<a href="https://colab.research.google.com/github/ernestoongaro/dbt-cloud-job-exporter-and-migrator/blob/main/dbt_Cloud_Jobs_Migrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is this for?
There are times where we need to move job definitions from:
* one dbt Cloud account to another
* one dbt Cloud project to another
* a Multi-tenant dbt Cloud account to a single-tenant one or vice versa

dbt Cloud does not have built-in import/export functionality through the UI, but it's possible to do through the API, so we do it here, via the API!

In [ ]:
!apt-get -qq update && apt-get -qq install python3.8 python3-pip jq
!python3.8 -m pip install git+https://github.com/data-mie/dbt-cloud-cli@main
from getpass import getpass

In [45]:
 
#Source dbt Cloud Project
import os
os.environ["DBT_CLOUD_API_TOKEN"] = getpass('API TOKEN:') #don't edit this - enter interactively
os.environ["DBT_CLOUD_ACCOUNT_ID"] = '4238'
os.environ["DBT_CLOUD_PROJECT_ID"] = '31070'
os.environ["DBT_CLOUD_HOST"] = 'cloud.getdbt.com'
os.environ["IMPORT_FILE_PATH"] = '/content/gdrive/My Drive/dbt_cloud_job_exports/' + os.environ["DBT_CLOUD_PROJECT_ID"]


API TOKEN:··········


In [46]:
%%bash 
 
job_list=(`dbt-cloud job list --project-id $DBT_CLOUD_PROJECT_ID | jq '.data[].id'`)
echo "The following jobs will be migrated:"
for job in ${job_list[@]}; do
  echo $job
done

The following jobs will be migrated:
25179
25185
37028


In [64]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!mkdir -p "$IMPORT_FILE_PATH"

Mounted at /content/gdrive


Export the jobs to file!

In [68]:
%%bash 
job_list=(`dbt-cloud job list --project-id $DBT_CLOUD_PROJECT_ID | jq '.data[].id'`)
for job in ${job_list[@]}; do
  dbt-cloud job export --job-id $job > "$IMPORT_FILE_PATH/${job}.json"
done

/content/gdrive/My Drive/dbt_cloud_job_exports/31070/25179.json
/content/gdrive/My Drive/dbt_cloud_job_exports/31070/25185.json
/content/gdrive/My Drive/dbt_cloud_job_exports/31070/37028.json


In [86]:
 
#Destination dbt Cloud Project
import os
os.environ["DEST_DBT_CLOUD_API_TOKEN"] = getpass('API TOKEN:') #don't edit this - enter interactively
os.environ["DEST_DBT_CLOUD_ACCOUNT_ID"] = '25898'
os.environ["DEST_DBT_CLOUD_PROJECT_ID"] = '69891'
os.environ["DEST_DBT_CLOUD_ENVIRONMENT_ID"] = '66688'
os.environ["DEST_DBT_CLOUD_HOST"] = 'cloud.getdbt.com'

API TOKEN:··········


In [87]:
%%bash 
job_list=(`ls "$IMPORT_FILE_PATH"`)
echo "The following jobs will be imported:"
for job in ${job_list[@]}; do
  echo $job
done

The following jobs will be imported:
25179.json
25185.json
37028.json


Now let's import, 🔔 careful, this will create new jobs for every time it's run!

In [111]:
%%bash 
job_list=(`ls "$IMPORT_FILE_PATH"`)
for job in ${job_list[@]}; do
  cat "$IMPORT_FILE_PATH"/${job} | jq '.environment_id = env.DEST_DBT_CLOUD_ENVIRONMENT_ID | .account_id = env.DEST_DBT_CLOUD_ACCOUNT_ID | .project_id = env.DEST_DBT_CLOUD_PROJECT_ID' \
   | dbt-cloud job import --dbt-cloud-host $DEST_DBT_CLOUD_HOST --api-token $DEST_DBT_CLOUD_API_TOKEN --account-id $DEST_DBT_CLOUD_ACCOUNT_ID | jq '"Job ID: " + (.data.id | tostring) + " - exited with code: " + (.status.code | tostring)'
done

"Job ID: 60605 - exited with code: 201"
"Job ID: 60606 - exited with code: 201"
"Job ID: 60607 - exited with code: 201"
